### Este paso sirve para detectar por inspección visual canales que tengan muchos artificios
La deteccion se hace 

1) Mediante observación del grafico de todos los LFPs

2) Mediante observacion de la distribucion de las derivadas del LFP (diferencial)

3) Finalmente sustrae el promedio de los canales válidos de cada canal, por lo tanto, cada canal queda referenciado al promedio

In [ ]:
%reset

In [2]:
import pickle
% pylab
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import signal
#normal charts inside notebooks
#%matplotlib inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


### Vamos a abrir el pickle generado por el codigo 01_ExtraerLFP

In [10]:
# colocar la ruta y el nombre del archivo pickle generado con el codigo 01_ExtraerLFP

# RAT 02
#fname = 'Raw_LFP_RAT02_SP_7_2017_01_11_18_46_49'
#ruta ='/home/romulo/DATA/Carolina/R02/RAT02_SP_7_2017_01_11_18_46_49/'

# RAT 05
#ruta = '/home/romulo/DATA/Carolina/C05/EC01_RC05_SP_10100uA_20171214/'
#fname = 'Raw_LFP_RC05_SP_10100uA_20171214_170009'

# RAT 03
#ruta = '/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_3500uA_20171215/'
#fname = 'Raw_LFP_EC01_RRR03_SP_3500uA_20171215'

#ruta = '/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_3500uA_L-Dopa_20171215/'
#fname = 'Raw_LFP_EC01_RRR03_SP_3500uA_L-Dopa_20171215'

#ruta='/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_4800uA_20171221/'
#fname='Raw_LFP_EC01_RRR03_SP_4800uA_20171221'

#ruta = '/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_1240uA_20171212/'
#fname = 'Raw_LFP_EC01_RRR03_SP_1240uA_20171212'

# RAT N03
#ruta = '/home/romulo/DATA/Carolina/N03/E01_RNA03_SP02_500mV_20170923/'
#fname = 'Raw_LFP_E01_RNA03_SP02_500mV_20170923'

# RAT 01
#ruta = '/home/romulo/DATA/Carolina/RR01/EC01_RR01_SP_6000uA_20171229/'
#fname = 'Raw_LFP_EC01_RR01_SP_6000uA_20171229'

#ruta='/home/romulo/DATA/Carolina/RR01/EC01_RRR01_SP_12600uA__20180103/'
#fname='Raw_LFP_EC01_RRR01_SP_12600uA__20180103'

#ruta = '/home/romulo/DATA/Marmoset/'
#fname = 'Raw_LFP_Deco_24102012_single_pulse_1Hz-01'


##CAMBIOS SERGIO

#RAT01
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RR01/EC01_RRR01_SP_12600uA__2018-01-03_13-09-47/'
#fname='Raw_LFP_EC01_RRR01_SP_12600uA__20180103'

#RAT02
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RAT02/RAT02_SP_7__2017-01-11_18-46-49/'
#fname='Raw_LFP_RAT02_SP_7__20170111_184649'

#RAT03
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RRR03/EC01_RRR03_SP_3500uA_2017-12-15_11-45-36/'
#fname='LFP_EC01_RRR03_SP_3500uA_20171215'

#RAT05
#ruta ='/home/sergio/Documentos/MemoriaPractII/DatosCaro/RC05/EC01_RC05_SP_10100uA_2017-12-14_17-00-09/'
#fname='LFP_EC01_RC05_SP_10100uA_20171214'

lfp = pickle.load(open( ruta+fname+'.pickle', "rb" ) )


# Changed channel areas nane to get differential referencing withinh the same area only for RAT02_SP_7_2017_01_11_18_46_49

Referenced to area / right M2 was divided in two groups for referencing: CH25-CH29 and CH21-CH24
19:23 and 23:28

### SOLO PARA LA RATA 2

In [ ]:
#de nuevo, solo para rata 2
lfp.Area[19:23] = 'rM2a'
lfp.Area[23:28] = 'rM2b'

In [11]:
lfp

[   Channel  Area OriginalSampleRate  SampleRate Use  \
 0      CH1   rVL              30000      1000.0   1   
 1     CH10   rM1              30000      1000.0   1   
 2     CH11   rM1              30000      1000.0   1   
 3     CH12  rSTR              30000      1000.0   1   
 4     CH13  rSTR              30000      1000.0   1   
 5     CH14  rSTR              30000      1000.0   1   
 6     CH15  rSTR              30000      1000.0   1   
 7     CH16   rVL              30000      1000.0   1   
 8     CH17  rVPL              30000      1000.0   1   
 9     CH18  rVPL              30000      1000.0   1   
 10    CH19   rS1              30000      1000.0   1   
 11     CH2   rVL              30000      1000.0   1   
 12    CH20   rS1              30000      1000.0   1   
 13    CH21  rSTN              30000      1000.0   1   
 14    CH22  rSTN              30000      1000.0   1   
 15    CH23   rPf              30000      1000.0   1   
 16    CH24   rPf              30000      1000.0

# Seleccionar los canales a no usar, primero graficar los lfps en el tiempo y la distribucion de las diferenciales

In [13]:
# Grafica todos los canales
N = 2 # ORDEN DEL FILTRO
#sr
sampleRate = lfp.SampleRate[0]
nyq = sampleRate/2.0
f1 = 0.1 # FRECUENCIAS DE CORTE
f2 = 75.0 # FRECUENCIAS DE CORTE
# band pass freqs expressed as Nyquist
Wn = np.array([f1/nyq, f2/nyq])
# design parameters for the butter filter
#Butter es un filtro pasabanda Butterworth. N es el orden del filtro, Wn el rango, analog se elige Falso
# para obtener un filtro digital. b es el denominador y a el numerador.  
b,a = signal.butter(N, Wn, btype='bandpass', analog=False)

#1) graficar todos los lfps en el tiempo
bar = 1000
fig, ax = subplots()
#sampleRate = lfp.SampleRate[0]
aux = len(lfp.LFP[0])
#numpy.arange([start, ]stop, [step, ]
t = arange(1/sampleRate,aux/sampleRate+1/sampleRate,1/sampleRate)

for i in range(0,len(lfp)):
    #Filtfilt is a filter that doesnt cause any phase distortions. padlen length of padding vector. None default
    x=signal.filtfilt(b, a, lfp.LFP[i], padlen=None) # with filter
    #x = lfp.LFP[i] # no filter
    ax.plot(t,x+bar*i)
    if i == 0:
        lowerlim = min(lfp.LFP[i]+bar*i)
    if i == len(lfp)-1:
        upperlim = max(lfp.LFP[i]+bar*i)
i = i+1
#ax.plot(t,average_lfp+bar*i)
ax.set_yticks(range(0,(i+1)*bar,bar))
ax.set_yticklabels(lfp.Area+lfp.Channel)
ax.set_xlim([t[0],t[-1]])
ax.set_ylim(lowerlim,upperlim)
plt.tight_layout()
#plt.savefig('RR01_All_Channels.png', bbox_inches='tight')
#plt.close('all')

In [ ]:
# Grafica por area
N = 2 # ORDEN DEL FILTRO
sr = lfp.SampleRate[0]
nyq = sr/2.0
f1 = 0.1 # FRECUENCIAS DE CORTE
f2 = 75.0 # FRECUENCIAS DE CORTE
# band pass freqs expressed as Nyquist
Wn = np.array([f1/nyq, f2/nyq])
# design parameters for the butter filter
b,a = signal.butter(N, Wn, btype='bandpass', analog=False)

bar = 1000

sampleRate = lfp.SampleRate[0]
aux = len(lfp.LFP[0])
t = arange(1/sampleRate,aux/sampleRate+1/sampleRate,1/sampleRate)


UniqAreas=unique(lfp.Area.tolist())
for i in UniqAreas:
    fig, ax = subplots()
    ind = np.where(lfp.Area==i)
    ind=ind[0]
    c=0
    #print i
    yticklabels = []
    for j in ind:
        print (j)
        #x=signal.filtfilt(b, a, lfp.LFP[j], padlen=None) # with filter
        x = lfp.LFP[j] # no filter
        ax.plot(t,x+bar*c)
        yticklabels.append(lfp.Channel[j]+'-'+lfp.Area[j])
        c=c+1
    ax.set_yticks(range(0,(c+1)*bar,bar))
    ax.set_yticklabels(yticklabels)
    ax.set_xlim([t[0],t[-1]])
    #ax.set_ylim(lowerlim,upperlim)
    plt.tight_layout()
    ax.set_title(i)
    #plt.savefig('RR01_By_Area_'+str(UniqAreas)+'.png', bbox_inches='tight')

In [5]:
close('all')

In [ ]:
print (len(yticklabels))
print(len(ind))

In [ ]:
ind = np.where(lfp.Area==i)
ind

In [ ]:
i

In [4]:
# 2)Grafica la distribucion de la derivada de la señal de LFP por canal
b = 1000
fig, ax_array = subplots(nrows=8,ncols=8)
for i in range(0,8):
    for j in range(0,8):
        ax = ax_array[i,j]
        if i*8+j < len(lfp):
            aux = lfp.LFP[i*8+j]
            ax.hist(diff(aux),bins=b, range =[-1000, 1000])
            ax.set_title(lfp.Channel[i*8+j])
            ax.set_ylim(0,20)       

# Put here channels you are not going to use, those will be removed from the data frame

In [6]:
# RAT 01
#Raw_LFP_EC01_RR01_SP_6000uA_20171229 BAD , DONT USE
# Raw_LFP_EC01_RRR01_SP_12600uA__20180103
#Channels2Remove = ['CH19', 'CH37', 'CH38']

# LFP_RAT02_SP_7_2017_01_11_18_46_49
#Channels2Remove=[]

# EC01_RRR03_SP_3500uA_20171215
#Channels2Remove=['CH2', 'CH7', 'CH10', 'CH15', 'CH27', 'CH29', 'CH31', 'CH40', 'CH53']

#RC05_SP_10100uA_20171214_170009
Channels2Remove=[]
#------------------------------------------------#
# EC01_RRR03_SP_1240uA_20171212
# Channels2Remove = ['CH25', 'CH28', 'CH60']
# EC01_RRR03_SP_3500uA_20171215
#Channels2Remove=['CH2', 'CH7', 'CH10', 'CH15', 'CH27', 'CH29', 'CH31', 'CH40', 'CH53']
# Channels2Remove = [] # none, they are clearly different between electrode 1 and 2
#  E01_RNA03_SP02_500mV_20170923
# Channels2Remove = [] # none, yet, recording seems to be highly contaminated with 50 Hz

In [7]:
# identify the indices of the channels you want to remove and remove the rows from the dataframe
indices = []
for i in Channels2Remove:
    ind = np.where(lfp.Channel==i)
    ind = int(ind[0])
    indices.append(ind)
original_len= len(lfp)
lfp.drop(indices,axis=0,inplace=True)
lfp.reset_index(inplace=True)
lfp.drop('index',axis=1,inplace=True)
print (original_len)
print (len(lfp))

64
64


# Referenciar (restar) el promedio de los canales válidos a cada canal válido

In [ ]:
# create receptor array for all valid LFPs
lfp_array = np.zeros( (len(lfp),len(lfp.LFP[0])) )
shape(lfp_array)
# channels are demeaned and zscored and put into receptor array to compute average LFP
for i in range(0,len(lfp)):
    ad = lfp.LFP[i] - np.mean(lfp.LFP[i]) # demeaning
    lfp_array[i,:]= (ad - np.mean(ad)) / np.std(ad)
    
average_lfp = np.mean(lfp_array,axis=0)

fig, ax=subplots()
ax.plot(average_lfp)
ax.set_title('Your average LFP')

# selected valid LFP are demeaned, zscored and the average demeaned zscored LFP is subtracted
# result is put back  in the panda dataframe

#compara un canal unico antes y despues de la referenciacion
fig, ax = subplots()
ad = lfp.LFP[0] - np.mean(lfp.LFP[0]) # demeaning
lfprawzscored =  (ad - np.mean(ad)) / np.std(ad)
ax.plot(lfprawzscored,'b')

for i in range(0,len(lfp)):
    print i
    ad = lfp.LFP[i] - np.mean(lfp.LFP[i]) # demeaning
    lfp.LFP[i] =  ((ad - np.mean(ad)) / np.std(ad)) - average_lfp
    
ax.plot(lfp.LFP[0],'r')

# Referenciar (restar) el promedio por electrodo canales (1-32) y luego canales (33-64)

In [ ]:
ChanList = { 'Electrode1':['CH1', 'CH2', 'CH3', 'CH4', 'CH5', 'CH6', 'CH7', 'CH8', 'CH9', 'CH10', 'CH11', 'CH12',
                           'CH13', 'CH14', 'CH15', 'CH16', 'CH17', 'CH18', 'CH19', 'CH20', 'CH21', 'CH22', 'CH23',
                           'CH24', 'CH25', 'CH26', 'CH27', 'CH28', 'CH29', 'CH30', 'CH31', 'CH32'],
             'Electrode2': ['CH33', 'CH34', 'CH35', 'CH36', 'CH37', 'CH38', 'CH39', 'CH40', 'CH41', 'CH42', 'CH43',
                            'CH44', 'CH45', 'CH46', 'CH47', 'CH48', 'CH49', 'CH50', 'CH51', 'CH52', 'CH53', 'CH54',
                            'CH55', 'CH56', 'CH57', 'CH58', 'CH59', 'CH60', 'CH61', 'CH62', 'CH63', 'CH64']
            }

for i in range(0,len(ChanList)):
    aux=ChanList[ChanList.keys()[i]]
    lfp_array=np.zeros( (len(aux) , len(lfp.LFP[0])) ) 
    for j in range(0,len(aux)):
        ind = np.where(lfp.Channel==aux[j])
        ind= int(ind[0])
        ad =lfp.LFP[ind]
        add = ad - np.mean(ad) # demeaning
 
        lfp_array[j,:]= (add - np.mean(add)) / np.std(add)
        
    average_lfp = np.mean(lfp_array,axis=0)
    for j in range(0,len(aux)):
        ad =lfp.LFP[ind]
        add = ad - np.mean(ad) # demeaning
        lfp.LFP[ind] = ((add - np.mean(add)) / np.std(add)) - average_lfp


# Referenciar (restar) con promedio de cada área 

In [8]:
AreaList=lfp.Area.tolist()
UniqAreas=unique(AreaList)
for i in UniqAreas:
    ind = np.where(lfp.Area==i)
    ind=ind[0]
    print (ind), (len(ind))
    lfp_array=np.zeros( (len(ind), len(lfp.LFP[0])) ) 
    lfp_array2=np.zeros( (len(ind), len(lfp.LFP[0])) ) 
    c=0
    
    for j in ind:  
        ad =lfp.LFP[j]
        add = ad - np.mean(ad) # demeaning
        lfp_array[c,:]= (add - np.mean(add)) / np.std(add) # z-scoring        
        c=c+1
    average_lfp = np.mean(lfp_array,axis=0)
    
    for j in ind:
        ad =lfp.LFP[j]
        add = ad - np.mean(ad) # demeaning
        lfp.LFP[j] = ((add - np.mean(add)) / np.std(add)) - average_lfp # substracting z-scores

[31 32 34 35 36 37]


/home/sergio/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[50 51 52 53]
[46 47 56 57 58]
[48 49 54]
[28 29 30 38 39 40 41]
[26 27 42]
[43 45 59 60]
[ 1  2 55 61 62 63]
[15 16 17 18]
[10 12 20 21 23]
[13 14 19]
[ 3  4  5  6 22 33 44]
[ 0  7 11]
[ 8  9 24 25]


In [9]:
# save proccessed LFP file

processing = 'referencing to zscored average of the area that electrodes'
file2save = ruta+'LFP_'+fname[8:]+'.pickle'
print (file2save)
answer = input('The file below will be saved. Save?(Y/n):')
if answer == 'n':
    print ('Saving canceled')
elif answer == 'Y':
    print ('Saving file to %s...' % (file2save))
    # Guarda el archivo

    pickle.dump([lfp,processing,average_lfp], open(file2save, "wb" ) ) 
else:
    print ('Saving canceled')

/home/sergio/Documentos/MemoriaPractII/DatosCaro/RC05/EC01_RC05_SP_10100uA_2017-12-14_17-00-09/LFP_EC01_RC05_SP_10100uA_20171214.pickle
The file below will be saved. Save?(Y/n):Y
Saving file to /home/sergio/Documentos/MemoriaPractII/DatosCaro/RC05/EC01_RC05_SP_10100uA_2017-12-14_17-00-09/LFP_EC01_RC05_SP_10100uA_20171214.pickle...


In [ ]:
len(lfp.LFP[0]) 

In [ ]:
ruta